In [1]:
%matplotlib ipympl
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl 
import matplotlib.animation as animation
from matplotlib import gridspec
import scipy.ndimage
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d.proj3d import proj_transform
import matplotlib.animation as anim
import pdb
#%matplotlib notebook



Bad key "text.kerning_factor" on line 4 in
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)


setattr(Axes3D, 'arrow3D', _arrow3D)



In [3]:
def plotEllipse(phi, angle_th, ax):
    
    ax.set_xlim(-1.2,1.2) 
    ax.set_ylim(-1.2,1.2)
    ax.axis('off')
    
    phi = np.deg2rad(phi)
    angle_th = np.deg2rad(angle_th)

    E1 = np.cos(angle_th)
    E2 = np.sin(angle_th)
    coord_max = max(E1,E2)

    # x, y values to be plotted  
    Ex = E1*np.sin(omega*t - k*z)/coord_max
    Ey = E2*np.sin(omega*t - k*z + phi) /coord_max

    E1 = np.cos(angle_th)
    E2 = np.sin(angle_th)

    # x, y values to be plotted  
    Ex = E1*np.sin(omega*t - k*z)/coord_max
    Ey = E2*np.sin(omega*t - k*z + phi) /coord_max

    
    ax.scatter(Ex, np.zeros(len(Ey)), c="blue", s=4)
    ax.scatter(np.zeros(len(Ex)), Ey, c="red", s=4)
    ax.scatter(Ex, Ey, c="green", s=4)
    
    return ax


In [4]:
def plotSphere(phi_deg, angle_th_deg,ax):

    phi = np.deg2rad(phi_deg)
    angle_th = np.deg2rad(angle_th_deg)
    
    E1 = np.cos(angle_th)
    E2 = np.sin(angle_th)
    #coord_max = max(E1,E2)

    
    tau_0 = 0.5*np.arctan2(np.tan(2.0*angle_th)*np.cos(phi),1.0)
    eps=1e-16
    #axis_ratio = 0
    #The trick is to rotate the x-axis onto the MAJ-axis
    #The Mathematics does not distinguish between x-axis 
    #and y-axis or the maj-axis and the minor-axis .
    if ((phi == 0.0) or (E1*E2 < eps)):
    #Linear polarisation
        tau_0 = np.rad2deg(angle_th)
        epsilon_0 = 0.0
        s_maj_axis = 1.0
        s_min_axis = 0.0
    elif(np.abs(phi)==np.pi/2):
        if(angle_th > np.pi/4.0):
            tau_0 = np.rad2deg(np.pi/2)
            s_maj_axis = E2
            s_min_axis = E1
        elif(angle_th <= np.pi/4.0):
            tau_0 = 0.0
            s_maj_axis = E1
            s_min_axis = E2
    elif((phi!=0.0) and (np.abs(phi)!=np.pi/2)):
        if(np.cos(phi) >= 0.0):
            if(angle_th <= np.pi/4.0):
                tau_0 = tau_0
            elif(angle_th > np.pi/4.0):
                tau_0 = np.pi/2.0 + tau_0 #tau becomes -ve in this case
        elif(np.cos(phi) < 0.0):
            if(angle_th > np.pi/4.0):
                tau_0 = tau_0 + np.pi/2.0
            elif(angle_th <= np.pi/4.0):
                tau_0 = np.pi + tau_0 #tau becomes -ve in this case
        
        #print("tau_0_2 ="+str(tau_0))
        #print("theta ="+str(angle_th))
        #print("phi ="+str(phi))

                    
        a2 = ((np.sin(angle_th)*np.cos(angle_th)*np.sin(phi))**2)/((np.sin(tau_0)*np.cos(angle_th))**2 + (np.cos(tau_0)*np.sin(angle_th))**2 - 2.0*np.sin(tau_0)*np.cos(tau_0)*np.sin(angle_th)*np.cos(angle_th)*np.cos(phi))
        b2 = ((np.sin(angle_th)*np.cos(angle_th)*np.sin(phi))**2)/((np.cos(tau_0)*np.cos(angle_th))**2 + (np.sin(tau_0)*np.cos(angle_th))**2 + 2.0*np.sin(tau_0)*np.cos(tau_0)*np.sin(angle_th)*np.cos(angle_th)*np.cos(phi))

        a = np.sqrt(a2)
        b = np.sqrt(b2)
        #print(a,b)
        tau_0 = np.rad2deg(tau_0)
        s_maj_axis = max(a,b)
        s_min_axis = min(a,b)
        if (s_min_axis <= eps):
            s_min_axis = eps

    if (phi >= 0): # by convention +phi => LCP
        axis_ratio = np.divide(s_maj_axis,s_min_axis)   # -phi => RCP
    elif (phi < 0): # on the PS, NP = LCP
        axis_ratio = np.divide(-s_maj_axis,s_min_axis)  #SP = RCP
    epsilon_0 = np.rad2deg(np.arctan2(1,axis_ratio))
    
    '''
    print("s_maj_axis = ",s_maj_axis)
    print("s_min_axis = ",s_min_axis)
    print("angle_th = ",angle_th)                                 
    print("axis ratio = ",np.divide(s_maj_axis,s_min_axis))
    print("E1 = ",E1)
    print("E2 = ",E2)
    print("E2/E1 = ",np.divide(E2,E1))
    print("phi_deg = ",phi_deg)
    print("tau_0 (linear PA) = ",tau_0) # 2*tau_0 = longitude in Poincare Sphere
    print("epsilon_0 (latitude in Poincare Sphere) = ",epsilon_0)     # 2*epsilon_0 = latitude in Poincare Sphere 
    '''
    
    #********** Define the Poincare Sphere ************
    R = 1
    n_tau = 20
    n_epsilon = 15 
    tau = np.linspace(0,180,n_tau)          # 2*tau = longitude
    tau = np.deg2rad(tau)

    epsilon = np.linspace(-45,45,n_epsilon)   # 2*epsilon = latitude
    epsilon = np.deg2rad(epsilon)
    x_par = np.zeros((n_tau,n_epsilon ))
    y_par = np.zeros((n_tau,n_epsilon ))
    z_par = np.zeros((n_tau,n_epsilon ))
    for j in range(0, n_epsilon):
        for i in range(0, n_tau):
            x_par[i,j] = R*np.cos(2*epsilon[j])*np.cos(2*tau[i])
            y_par[i,j] = R*np.cos(2*epsilon[j])*np.sin(2*tau[i])
            z_par[i,j] = R*np.sin(2*epsilon[j])

    tau_0_rad = np.deg2rad(tau_0)
    epsilon_0_rad = np.deg2rad(epsilon_0)
    x_par_0 = R*np.cos(2*epsilon_0_rad)*np.cos(2*tau_0_rad)
    y_par_0 = R*np.cos(2*epsilon_0_rad)*np.sin(2*tau_0_rad)
    z_par_0 = R*np.sin(2*epsilon_0_rad)
    
    
    
    #fig = plt.figure(figsize=(8,8))
    #ax = fig.add_subplot(projection='3d')

    ax.plot_wireframe(x_par, y_par, z_par,alpha=0.15)
    
    ax.arrow3D(0,0,0,x_par_0, y_par_0, z_par_0,color="r",mutation_scale=30,ec ='red',fc='red')

    
    x_eq = np.where(z_par==0,x_par,0)
    y_eq = np.where(z_par==0,y_par,0)
    z_eq = np.where(z_par==0,z_par,0)
    ax.plot_surface(x_eq, y_eq, z_eq, rstride=1, cstride=1, alpha=0.1)

    ax.scatter(0,0,0,c="k",s=100)

    #ax.set_box_aspect(aspect=(1, 1, 1))


    ax.set_xlabel("Stokes Q")
    ax.set_ylabel("Stokes U")
    ax.set_zlabel("Stokes V")

    # Get rid of colored axes planes
    # First remove fill
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False

    # Now set color to white (or whatever is "invisible")
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    ax.xaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})
    ax.yaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})
    ax.zaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})

    lat = 2*epsilon_0
    if(lat > 90):
        lat = lat - 360
    
    return ax, tau_0, lat, x_par_0, y_par_0, z_par_0


In [5]:
#testData = np.random.rand(100,2)-[0.5, 0.5]

numframes = 90

#phi_deg = np.linspace(0,90,numframes) #phase difference between Ex and Ey
#angle_th = np.linspace(0,90,numframes)  #pol position angle

#phis = np.deg2rad(phi_deg)
#angle_ths = np.deg2rad(angle_th)

#constants
c = 3e8
z = 0

omega = 10
nu = omega/(2*np.pi)
Lambda = c/nu
T = 1/nu # Time-Period of the wave
ntCycles=4
n_t = ntCycles*100
t_T = np.linspace(0,ntCycles*T,n_t)
k = 2*np.pi/Lambda

t =  t_T
z = 0



In [12]:
phi = -119.9
angle_th = 50.0
cmap = matplotlib.cm.get_cmap('Spectral')

col = cmap(0.5)
fig = plt.figure(figsize=(11,5))
ax2 = fig.add_subplot(1,2,2, projection='3d')
ax2, psi, twochi, x_par_0, y_par_0, z_par_0 = plotSphere(phi, angle_th,ax2)
ax2.scatter(x_par_0, y_par_0, z_par_0, c=col)


ax2.view_init(elev=20, azim=295)
ax2.set_xlim(-1.2,1.2)
ax2.set_ylim(-1.2,1.2)
ax2.set_zlim(-1.2,1.2)
ax2.xaxis.set_major_locator(plt.MaxNLocator(3))
ax2.yaxis.set_major_locator(plt.MaxNLocator(3))
ax2.zaxis.set_major_locator(plt.MaxNLocator(3))
ax2.margins(x=0)

#plt.show()

#fig = plt.figure(figsize=(8,8))
ax1 = fig.add_subplot(1,2,1)
ax1 = plotEllipse(phi, angle_th, ax1)
ax1.text(-1.1,1.1,fr"$\theta$ = {angle_th:.1f}°", fontsize=20)
ax1.text(-1.1,0.95,fr"$\phi$ = {phi:.1f}°", fontsize=20)
ax1.text(0.5,1.1,fr"$2\psi$ (lon) = {2*psi:.1f}°", fontsize=20)
ax1.text(0.5,0.95,fr"2$\chi$ (lat) = {twochi:.1f}°", fontsize=20)
plt.show()
plt.tight_layout()
fig.savefig("test.png",bbox_inches='tight')


FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.


In [ ]:
i=400
(270+i)%360

In [14]:
phis = np.linspace(-179,179,81) #phase difference between Ex and Ey
thetas = np.linspace(0,90,21)  #pol position angle

cmap = matplotlib.cm.get_cmap('Spectral')


index=0
posx = []
posy = []
posz = []
colours = []
for i in range(0, len(thetas)):
    for j in range(0, len(phis)):
        phi = phis[j]
        angle_th = thetas[i]
        #az = (270+index)%360
        az=300
        name = str(index).zfill(5)+f"_{phi:.0f}_{angle_th:.0f}"        
        index = index+1
        fig = plt.figure(figsize=(22,10))
        ax2 = fig.add_subplot(1,2,2, projection='3d')

        ax2, psi, twochi, x_par_0, y_par_0, z_par_0= plotSphere(phi, angle_th,ax2)
        posx.append(x_par_0)
        posy.append(y_par_0)
        posz.append(z_par_0)
        #col = cmap(i/len(thetas))
        col = cmap(i/len(thetas))
        colours.append(col)
        ax2.scatter(posx, posy, posz, c = colours)
        ax2.view_init(elev=30, azim=az)
        ax2.set_xlim(-1.2,1.2)
        ax2.set_ylim(-1.2,1.2)
        ax2.set_zlim(-1.2,1.2)
        ax2.xaxis.set_major_locator(plt.MaxNLocator(3))
        ax2.yaxis.set_major_locator(plt.MaxNLocator(3))
        ax2.zaxis.set_major_locator(plt.MaxNLocator(3))
         

        #fig.savefig(f"sphere_anim/{name}_sphere.png")
        #plt.close()
        #fig = plt.figure(figsize=(8,8))
        ax1 = fig.add_subplot(1,2,1)
        #ax1 = fig.add_subplot(1,1,1)
        ax1 = plotEllipse(phi, angle_th, ax1)
        ax1.text(-1.1,1.1,fr"$\theta$ = {angle_th:.1f}°", fontsize=20)
        ax1.text(-1.1,0.95,fr"$\phi$ = {phi:.1f}°", fontsize=20)
        ax1.text(0.5,1.1,fr"$2\psi$ (lon) = {2*psi:.1f}°", fontsize=20)
        ax1.text(0.5,0.95,fr"2$\chi$ (lat) = {twochi:.1f}°", fontsize=20)
        fig.savefig(f"sphere_anim_consttheta/{name}.png")
        plt.close()

        
for i in range(0, 360):
    phi = phis[-1]
    angle_th = thetas[-1]
    az = (az+1)%360
    #az=300
    name = str(index).zfill(5)+f"_{phi:.0f}_{angle_th:.0f}"        
    index = index+1
    fig = plt.figure(figsize=(22,10))
    ax2 = fig.add_subplot(1,2,2, projection='3d')

    ax2, psi, twochi, x_par_0, y_par_0, z_par_0= plotSphere(phi, angle_th,ax2)
    #posx.append(x_par_0)
    #posy.append(y_par_0)
    #posz.append(z_par_0)
    #col = cmap(i/len(thetas))
    #colours.append(col)
    ax2.scatter(posx, posy, posz, c = colours)
    ax2.view_init(elev=30, azim=az)
    ax2.set_xlim(-1.2,1.2)
    ax2.set_ylim(-1.2,1.2)
    ax2.set_zlim(-1.2,1.2)
    ax2.xaxis.set_major_locator(plt.MaxNLocator(3))
    ax2.yaxis.set_major_locator(plt.MaxNLocator(3))
    ax2.zaxis.set_major_locator(plt.MaxNLocator(3))


    #fig.savefig(f"sphere_anim/{name}_sphere.png")
    #plt.close()
    #fig = plt.figure(figsize=(8,8))
    ax1 = fig.add_subplot(1,2,1)
    #ax1 = fig.add_subplot(1,1,1)
    ax1 = plotEllipse(phi, angle_th, ax1)
    ax1.text(-1.1,1.1,fr"$\theta$ = {angle_th:.1f}°", fontsize=20)
    ax1.text(-1.1,0.95,fr"$\phi$ = {phi:.1f}°", fontsize=20)
    ax1.text(0.5,1.1,fr"$2\psi$ (lon) = {2*psi:.1f}°", fontsize=20)
    ax1.text(0.5,0.95,fr"2$\chi$ (lat) = {twochi:.1f}°", fontsize=20)
    fig.savefig(f"sphere_anim_consttheta/{name}.png")
    plt.close()

FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


FigureCanvasNbAgg()

...

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [12]:
az=300
for i in range(0, 360):
    phi = phis[-1]
    angle_th = thetas[-1]
    az = (az+1)
    print(az)

301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550


In [13]:
index=1701
for i in range(0, 360):
    phi = phis[-1]
    angle_th = thetas[-1]
    az = (az+1)%360
    #az=300
    name = str(index).zfill(5)+f"_{phi:.0f}_{angle_th:.0f}"        
    index = index+1
    fig = plt.figure(figsize=(22,10))
    ax2 = fig.add_subplot(1,2,2, projection='3d')

    ax2, psi, twochi, x_par_0, y_par_0, z_par_0= plotSphere(phi, angle_th,ax2)
    #posx.append(x_par_0)
    #posy.append(y_par_0)
    #posz.append(z_par_0)
    #col = cmap(i/len(thetas))
    #colours.append(col)
    ax2.scatter(posx, posy, posz, c = colours)
    ax2.view_init(elev=30, azim=az)
    ax2.set_xlim(-1.2,1.2)
    ax2.set_ylim(-1.2,1.2)
    ax2.set_zlim(-1.2,1.2)
    ax2.xaxis.set_major_locator(plt.MaxNLocator(3))
    ax2.yaxis.set_major_locator(plt.MaxNLocator(3))
    ax2.zaxis.set_major_locator(plt.MaxNLocator(3))


    #fig.savefig(f"sphere_anim/{name}_sphere.png")
    #plt.close()
    #fig = plt.figure(figsize=(8,8))
    ax1 = fig.add_subplot(1,2,1)
    #ax1 = fig.add_subplot(1,1,1)
    ax1 = plotEllipse(phi, angle_th, ax1)
    ax1.text(-1.1,1.1,fr"$\theta$ = {angle_th:.1f}°", fontsize=20)
    ax1.text(-1.1,0.95,fr"$\phi$ = {phi:.1f}°", fontsize=20)
    ax1.text(0.5,1.1,fr"$2\psi$ (lon) = {2*psi:.1f}°", fontsize=20)
    ax1.text(0.5,0.95,fr"2$\chi$ (lat) = {twochi:.1f}°", fontsize=20)
    fig.savefig(f"sphere_anim_constphi/{name}.png")
    plt.close()

FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


FigureCanvasNbAgg()

...

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


FigureCanvasNbAgg()

/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in true_divide
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
